In [ ]:
from ridsans.sansdata import *

In [ ]:
Q = 4
sans_file = SansData(
    f"sample-data/transmission_2mm_perspex_boronglass_noBeamstop_Q{Q}.mpa",
    log_process=True,
    rebin=False,
)


def plot_image(data, label):
    extent = cropped_extent
    plt.imshow(data, cmap="viridis", extent=extent, aspect="auto")
    plt.colorbar(label=label)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid()
    plt.show()

In [ ]:
I = sans_file.I
plot_I(I, True)

In [ ]:
I = sans_file.I[250:-250, 250:-250]
plot_I(I, True)

In [ ]:
compute_spectrum = lambda x: np.log(
    np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift(x))))
)
compute_inverse = lambda x: np.abs(
    np.fft.ifftshift(np.fft.ifft2(np.fft.ifftshift(np.exp(x))))
)

spectrum = compute_spectrum(sans_file.I)
plot_image(spectrum, "I spectrum")

In [ ]:
data, label = spectrum, "I spectrum"
offset = 147
x0 = 3 + 233 + active_w_pixels / 2
y0 = 0 + 233 + active_w_pixels / 2
extent = [233, 233 + active_w_pixels, 233, 233 + active_w_pixels]
plt.imshow(data, cmap="viridis", extent=extent, aspect="auto")
plt.colorbar(label=label)
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

# Add symmetric crosses
plt.scatter(x0, y0 + offset, color="red", marker="+", s=100, label="Cross 1")
plt.scatter(x0, y0, color="green", marker="+", s=100, label="Cross 1")
plt.scatter(x0, y0 - offset, color="blue", marker="+", s=100, label="Cross 2")

plt.legend()
plt.show()

In [ ]:
def rebin2d(arr, n):
    rows, cols = arr.shape
    if rows % n != 0 or cols % n != 0:
        print(f"Array dimensions are not divisible by {n}, trimming remainder")
        x_lim = (rows // n) * n
        y_lim = (cols // n) * n
        arr = arr[:x_lim, :y_lim]
    return arr.reshape(rows // n, n, cols // n, n).sum(axis=(1, 3))


I_rebin = rebin2d(sans_file.I, 4)
plot_I(I_rebin, True)

# Generate placeholder efficiency map

In [ ]:
placeholder_efficiency = np.ones((round(active_w_pixels / 4), round(active_w_pixels / 4)), dtype=np.float64)
print(placeholder_efficiency)
print(placeholder_efficiency.shape)
print(placeholder_efficiency.dtype)
np.savetxt("pixel-efficiency.txt.gz", placeholder_efficiency)

In [ ]:
568 / 4

In [ ]:
I = I_rebin[20:-20, 20:-20]
plot_I(I, True)

In [ ]:
spectrum = compute_spectrum(I_rebin)
plot_image(spectrum, "I spectrum")

In [ ]:
from scipy.ndimage import gaussian_filter

I = sans_file.I
dI = sans_file.dI

d_rel = dI / I
sigma = 3.0  # Standard deviation of the Gaussian kernel
# Apply Gaussian smoothing to the data
smoothed_data = gaussian_filter(I, sigma=sigma)
smoothed_uncertainty = gaussian_filter(dI**2, sigma=sigma)
propagated_uncertainty = np.sqrt(dI) + 0.00001
plot_I(smoothed_data, True)

normalized_I = I / smoothed_data
plot_image(normalized_I, "relative efficiency")
# print(propagated_uncertainty)

In [ ]:
plot_projections(I / smoothed_data)

In [ ]:
from scipy.ndimage import gaussian_filter

I = I_rebin
dI = np.sqrt(I_rebin)
sigma = 1.0  # Standard deviation of the Gaussian kernel
# Apply Gaussian smoothing to the data
smoothed_data = gaussian_filter(I, sigma=sigma)
smoothed_uncertainty = gaussian_filter(dI**2, sigma=sigma)
propagated_uncertainty = np.sqrt(dI) + 0.00001
plot_I(smoothed_data, True)

normalized_I = I / smoothed_data
dnormalized_I = dI / smoothed_data
plot_image(normalized_I, "relative efficiency")
# print(propagated_uncertainty)

In [ ]:
# Flatten the 2D image into a 1D array for the histogram
flattened_image = normalized_I.ravel()

# Compute the number of bins (use the square root of the total number of values as a heuristic)
num_bins = int(np.sqrt(flattened_image.size))

# Plot the histogram
plt.figure(figsize=(8, 6))
plt.hist(flattened_image, bins=num_bins, color="blue", alpha=0.7, edgecolor="black")
plt.title("Histogram of Image Values")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.xlim(left=0)
plt.show()

In [ ]:
# Flatten the 2D image into a 1D array for the histogram
flattened_image = normalized_I.ravel()

In [ ]:
plot_projections(I / smoothed_data)

In [ ]:
from scipy.stats import poisson

# Parameters
lambda_ = 5  # Mean number of events

# Compute the Poisson distribution
x = np.arange(0, 15)  # Range of values
pmf = poisson.pmf(x, lambda_)  # Probability mass function

# Plot the distribution
plt.bar(x, pmf, alpha=0.6)
plt.title("Poisson Distribution")
plt.xlabel("Number of Events")
plt.ylabel("Probability")
plt.show()

In [ ]:
counts = I.ravel()
min_c, max_c = np.min(counts), np.max(counts)
min_c, max_c

# Rebinning

In [ ]:
compute_spectrum = lambda x: np.log(
    np.abs(np.fft.fftshift(np.fft.fft2(np.fft.fftshift(x))))
)
compute_inverse = lambda x: np.abs(
    np.fft.ifftshift(np.fft.ifft2(np.fft.ifftshift(np.exp(x))))
)

spectrum = compute_spectrum(I)
normalized_spectrum = compute_spectrum(normalized_I)
plot_image(spectrum, "I spectrum")
plot_image(normalized_spectrum, "Normalized I spectrum")

## Characterize active detector region dimensions

In [ ]:
# Helper function for indexing a rectangle from an array
def rect_slice(center, size):
    x, y = center
    return (slice(y - size // 2, y + size // 2), slice(x - size // 2, x + size // 2))

beamstop_x, beamstop_y = 1024 - 500, 520
beamstop_w, beamstop_h = 76, 76
flood_intensities_full = sans_file.raw_intensity.astype(float)
flood_intensities_full[rect_slice((beamstop_x, beamstop_y), beamstop_w)] = 0
flood_intensities = flood_intensities_full
nonzero_indices = np.argwhere(flood_intensities_full >= 6)


min_x, min_y = nonzero_indices.min(axis=0)
max_x, max_y = nonzero_indices.max(axis=0)
min_x += 18
max_x += 4
max_x -= 2
min_y -= 2
centre_x = (max_x + min_x) / 2
centre_y = (max_y + min_y) / 2

# The number of bins corresponding to the number of active pixels
w = round(active_w_pixels / 4)
h = round(active_w_pixels / 4)
print(f"Min x, y: ({min_x}, {min_y})")
print(f"Max x, y: ({max_x}, {max_y})")
print(f"Centre x,y: ({centre_x}, {centre_y})")
print(f"Active pixels W x H: {w} x {h}")

plt.figure()
# extent = [550//2, 1024-550//2, 550//2, 1024-550//2]
extent = [0, 1024, 0, 1024]
plt.imshow(flood_intensities_full, cmap="viridis", extent=extent, aspect="auto")
plt.colorbar()
plt.xlabel("x")
plt.ylabel("y")
plt.axvline(min_x, color="red", linestyle="--")
plt.axvline(max_x, color="red", linestyle="--")
plt.axhline(min_y, color="red", linestyle="--")
plt.axhline(max_y, color="red", linestyle="--")
plt.grid()
plt.show()

## Compute pixel sizes and locations

In [ ]:
W_eff = 0.60  # m
N_pixels_eff = w
pixel_size = W_eff / N_pixels_eff  # m
print(f"Pixel size: {pixel_size} m")
half_pixel = pixel_size / 2
print(f"Half pixel size: {half_pixel} m")
N_pixels = w
W = pixel_size * N_pixels
print(f"Full width: {W} m")

In [ ]:
# Compute pixel centers
def compute_unit_centers(width, unit_width, unit_count):
    return unit_width / 2 + np.arange(0, unit_count) * unit_width - width / 2


pixel_x = compute_unit_centers(W, pixel_size, N_pixels_eff)
pixel_x

## Generate instrument definition file from template

In [ ]:
file_path = "RIDSANS_Definition_template.xml"

with open(file_path, "r") as file:
    file_content = file.read()
# print(file_content)

In [ ]:
value_map = {
    "PIXEL_SIZE": pixel_size,
    "HALF_PIXEL_SIZE": half_pixel,
    "PIXEL_START_X": pixel_x[0],
    "PIXEL_START_Y": pixel_x[0],
    "DETECTOR_PIXELS_X": N_pixels_eff,
    "DETECTOR_PIXELS_Y": N_pixels_eff,
}
for k, v in value_map.items():
    file_content = file_content.replace("$" + k, str(v))
file_content

In [ ]:
file_path = "RIDSANS_Definition.xml"

with open(file_path, "w") as file:
    file.write(file_content)

## Instrument definition loading test

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mantid.simpleapi import *

# create sample workspace
ws1 = CreateSampleWorkspace()
inst1 = ws1.getInstrument()
print(
    "Default workspace has instrument: {0} with {1} parameters".format(
        inst1.getName(), len(inst1.getParameterNames())
    )
)
ws2 = CreateSampleWorkspace()
mon2 = LoadInstrument(ws2, FileName="RIDSANS_Definition.xml", RewriteSpectraMap=True)
inst2 = ws2.getInstrument()
di2 = ws2.detectorInfo()
ci2 = ws2.componentInfo()
print("Workspace {0} has instrument: {1}".format(ws2.name(), inst2.getName()))
print(
    "Instrument {0} has {1} components, including {2} monitors and {3} detectors".format(
        inst2.getName(), ci2.size(), len(mon2), di2.size()
    )
)

## Compute relative pixel efficiencies from perspex measurement
Note: the [DetectorFloodWeighting](https://docs.mantidproject.org/nightly/algorithms/DetectorFloodWeighting-v1.html) algorithm appears to do the same and uses the mean technique rather than an absolute.

In [ ]:
# Compute pixel efficiency from flood measurement
relative_efficiency = False
if relative_efficiency:
    # Compensate for the fact that the beamstop square is not considered in the average
    area_factor = 1 / (1 - beamstop_w * beamstop_h / N_eff_pixel**2)
    print(area_factor)

    # Idea: compute relative pixel efficiency based on the average pixel counts over the given area
    mu_flood = (
        np.mean(flood_intensities[rect_slice((512, 512), N_eff_pixel)]) * area_factor
    )
    flood_norm = flood_intensities / mu_flood
else:
    # This assumes that the best pixel has efficiency 1? Also not ideal
    max_intensity = np.max(flood_intensities[rect_slice((512, 512), N_eff_pixel)])
    flood_norm = flood_intensities / max_intensity
    dflood_norm = np.sqrt(flood_intensities) / max_intensity
    # flood_norm[flood_norm<=0] = 1
    print(
        f"Mean factor: {np.mean(flood_norm[rect_slice((512, 512), N_eff_pixel)])}, mean error {np.mean(dflood_norm[rect_slice((512, 512), N_eff_pixel)])}"
    )

plot_image(flood_norm, "Relative efficiency")

## Plot integrals along $x,y$ to see relevance

In [ ]:
# Integrated over x, function of y
proj_y = np.sum(flood_norm, axis=1)
y_range = np.arange(0, 1024, 1)
plt.plot(y_range, proj_y)
plt.show()

# Integrated over y, function of x
proj_x = np.sum(flood_norm, axis=0)
x_range = np.arange(0, 1024, 1)
plt.plot(x_range, proj_x)
# plt.xlim((400,600))

## Histogram of pixel efficiencies
Verify that the statistics of efficiencies are reasonable: no extraordinarily high values etc.

In [ ]:
# Flatten the 2D array to a 1D array
active_flood = flood_norm[rect_slice((512, 512), N_eff_pixel)]
flat_array = (active_flood).flatten()

# Plot histogram
plt.hist(flat_array, bins=36, edgecolor="black")
plt.xlabel("Pixel efficiency")
plt.ylabel("Frequency")
plt.title("Histogram of efficiency values")
plt.show()

## Synthesize full efficiency map (temporary)
Currently, no flood measurement is available that can characterize the full detector so make up the values in the beamstop area for now by copying other values. NOT FOR USE IN ACTUAL DATA REDUCTIONS.

In [ ]:
flood_norm_synth = np.copy(flood_norm)
offset = 15
flood_norm_synth[rect_slice((beamstop_x, beamstop_y), beamstop_w + offset)] = (
    flood_norm_synth[
        rect_slice(
            (beamstop_x, beamstop_y + beamstop_w + offset + 10), beamstop_w + offset
        )
    ]
)
plot_image(flood_norm_synth, "Relative efficiency")

In [ ]:
np.savetxt("pixel-efficiency.txt.gz", flood_norm_synth[233:801, 233:801])